In [ ]:
import csv 
import nltk
import pandas as pd
import random
from nltk.corpus import *
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

wordlemmatizer = WordNetLemmatizer()

texts = pd.read_csv('SMSSpamCollection.tsv',sep="\t", quoting=csv.QUOTE_NONE, names=["label", "message"])


words = [wordlemmatizer.lemmatize(m.lower())  for w in texts.message for m in nltk.word_tokenize(w) if m not in stopwords and m.isalpha()]  
all_words = nltk.FreqDist(w for w in words)
featuresWord = all_words.most_common()
featuresWord = [w for (w,freq) in featuresWord]
featuresWord = featuresWord[:2000] 


def features(document): 
    document_words = set(document) 
    features = {}
    for word in featuresWord:
        features['contains({})'.format(word)] = (word in document_words) 
    return features


ham = []
for i in range(5570):
    word = texts.message[i]
    label = texts.label[i]
    if(label == 'ham' and i != 0):
        ham.append( (list(nltk.word_tokenize(word)), label))
#documents[5]

spam = []
for i in range(5570):
    word = texts.message[i]
    label = texts.label[i]
    if(label == 'spam' and i != 0):
        spam.append( (list(nltk.word_tokenize(word)), label))
    

#documents = ham + spam
#len(documents)
#random.shuffle(documents)
featuresetsham = [(features(d),c) for (d,c) in ham]
featuresetsspam = [(features(d),c) for (d,c) in spam]




train_set , test_set = featuresetsham[:3860] + featuresetsspam[:596]  ,  featuresetsham[3860:] + featuresetsspam[596:] 

classifier = nltk.NaiveBayesClassifier.train(train_set)
#classifier = nltk.NaiveBayesClassifier.train(trainham)

print ('Accuracy ' ,nltk.classify.accuracy(classifier, test_set))
print("------------------------------------------")
classifier.show_most_informative_features(10) 
